In [1]:
from pprint import pprint

import numpy as np
import pandas as pd

In [2]:
root = "/Users/runlanlana/Documents/2023_Spring/MIT6.C85_Interactive Data Visualization and Society/Data Client Project/"

In [3]:
main_df = pd.read_csv(root + "dataset-1_central-american-survey/main_table.csv", index_col = 0)
display(main_df.head(3))

main_df.info(verbose = True, show_counts = True)

,start,end,note_confidenciality,organizacion,survey_pid,country,departamento,municipio,neighborhood,rural_urban,...,_tags,rsp_id,family_type_reduced,migrated_yn,dp_ratio,income_per_capita,monthly_expenditure,exp_food_pct,income_sources,migrated_yn_5
0,2021-05-13T09:29:28.324-06:00,2021-05-13T09:59:31.341-06:00,NaN,1,34,SLV,1193,15748,Condominios delgado,2,...,NaN,4998,3,0,0.500000,3.666667,301.0,0.282392,3,0
1,2021-05-13T08:44:14.642-06:00,2021-05-13T09:05:10.406-06:00,NaN,1,34,SLV,1193,15748,Condominios santa maria,2,...,NaN,4997,1,0,0.500000,166.666667,305.0,0.491803,3,0
2,2021-05-13T08:10:59.086-06:00,2021-05-13T08:43:16.923-06:00,NaN,1,34,SLV,1193,15748,Condominios santa maria,2,...,NaN,4996,3,0,0.333333,75.000000,234.0,0.427350,3,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 455 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    start                         4998 non-null   object 
 1    end                           4998 non-null   object 
 2    note_confidenciality          0 non-null      float64
 3    organizacion                  4998 non-null   int64  
 4    survey_pid                    4998 non-null   int64  
 5    country                       4998 non-null   object 
 6    departamento                  4998 non-null   object 
 7    municipio                     4998 non-null   object 
 8    neighborhood                  4998 non-null   object 
 9    rural_urban                   4998 non-null   int64  
 10   note_consent_1                0 non-null      float64
 11   note_consent_2                0 non-null      float64
 12   note_consent_3                0 non-null      

In [4]:
# get municipal labels
municipal_labels = pd.read_csv(root + "utilities/municipio_labels.csv")
display(municipal_labels.head(3))
# merge with main
main_df = main_df.merge(municipal_labels, how = 'left', 
                   on = 'municipio')

,municipio,municipio_label
0,GT1612,Cahabon
1,GT1614,Chahal
2,GT1613,Chisec


### Clean HH size

There are 23 hhs indicating size of 0. prob by mistaken. Refill the values based on the type of family (`tipo_familia`)

In [5]:
main_df[main_df['hh_size'] == 0]['tipo_familia'].value_counts()

9    12
1     6
2     3
8     1
4     1
Name: tipo_familia, dtype: int64

In [6]:
print(f"Number of HH with size 0: {len(main_df[main_df['hh_size'] == 0])}")
# only one person
main_df.loc[(main_df['hh_size'] == 0) & (main_df['tipo_familia'] == 9), 'hh_size'] = 1
# biparental
main_df.loc[(main_df['hh_size'] == 0) & (main_df['tipo_familia'] == 1), 'hh_size'] = 3 
# single parent - mother
main_df.loc[(main_df['hh_size'] == 0) & (main_df['tipo_familia'] == 2), 'hh_size'] = 2
# reconstituted
main_df.loc[(main_df['hh_size'] == 0) & (main_df['tipo_familia'] == 4), 'hh_size'] = 3
# childless union
main_df.loc[(main_df['hh_size'] == 0) & (main_df['tipo_familia'] == 8), 'hh_size'] = 2

Number of HH with size 0: 23


In [7]:
main_df['staying'] = ~(main_df['mig_motivo_stay'].isna())
main_df['staying'] = main_df['staying'].map({True: "Yes", False: "No / Not Sure"})

main_df['migrating_ext_intent_label'] = main_df['mig_ext_intention'].map({
    0: "Not migrate",
    1: "Migrate",
    99: "Not sure"
})
main_df['migrating_int_intent_label'] = main_df['mig_ext_intention'].map({
    0: "Not migrate",
    1: "Migrate",
    99: "Not sure"
})

# Reasons to migrate externally

In [8]:
cols_to_keep = [f'mig_ext_pref_motivo/{n}' for n in range(1, 17)] + ['mig_ext_pref_motivo/99']
mig_ext_motivo_df = main_df[['mig_ext_intention', 'mig_ext_plan_12months'] + cols_to_keep]
mig_ext_motivo_df

,mig_ext_intention,mig_ext_plan_12months,mig_ext_pref_motivo/1,mig_ext_pref_motivo/2,mig_ext_pref_motivo/3,mig_ext_pref_motivo/4,mig_ext_pref_motivo/5,mig_ext_pref_motivo/6,mig_ext_pref_motivo/7,mig_ext_pref_motivo/8,mig_ext_pref_motivo/9,mig_ext_pref_motivo/10,mig_ext_pref_motivo/11,mig_ext_pref_motivo/12,mig_ext_pref_motivo/13,mig_ext_pref_motivo/14,mig_ext_pref_motivo/15,mig_ext_pref_motivo/16,mig_ext_pref_motivo/99
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4994,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4995,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# mig_ext_motivo_df_pct = mig_ext_motivo_df[mig_ext_motivo_df['mig_ext_intention'] == 1].mean().to_frame(name = 'percentage')
# mig_ext_motivo_df_pct.drop(index = ['mig_ext_intention', 'mig_ext_plan_12months'], inplace = True)
# mig_ext_motivo_df_pct.reset_index(inplace = True, names = 'motivation_choice')
# mig_ext_motivo_df_pct['motivation_choice'] = mig_ext_motivo_df_pct['motivation_choice'].str.replace("mig_ext_pref_motivo/", "")
# mig_ext_motivo_df_pct['percentage'] = mig_ext_motivo_df_pct['percentage'] * 100

choice_labels = {"1": 'Search for a better job',
                 "2": "Unemployment",
                 "3": "Deterioration of livelihoods due to natural hazards",
                 "4": "By the direct impact of a natural hazard",
                 "5": "Due to loss of land (use change)",
                 "6": "Lack of money for food",
                 "7": "Lack of money for other basic needs",
                 "8": "To send remittances",
                 "9": "For study",
                 "10": "Domestic violence",
                 "11": "Unsafety",
                 "12": "Family reunification",
                 "13": "For cultural reasons or custom",
                 "14": "For healthcare",
                 "15": "Adventure Tourism",
                 "16": "Other",
                 "99": "Not sure"
                 }
# mig_ext_motivo_df_pct['motivation'] = mig_ext_motivo_df_pct['motivation_choice'].map(choice_labels)
# mig_ext_motivo_df_pct.to_csv("./data/mig_ext_motivat_pct.csv", index = False)
# mig_ext_motivo_df_pct


In [12]:
mig_ext_motivo_df_pct_by_plan = mig_ext_motivo_df[mig_ext_motivo_df['mig_ext_intention'] == 1].groupby('mig_ext_plan_12months').mean()

mig_ext_motivo_df_pct_by_plan.drop(columns = ['mig_ext_intention'], inplace = True)
mig_ext_motivo_df_pct_by_plan.reset_index(inplace = True)
mig_ext_motivo_df_pct_by_plan = mig_ext_motivo_df_pct_by_plan.melt(id_vars = 'mig_ext_plan_12months', var_name = 'motivation_choice', value_name = 'percentage')
mig_ext_motivo_df_pct_by_plan['motivation_choice'] = mig_ext_motivo_df_pct_by_plan['motivation_choice'].str.replace("mig_ext_pref_motivo/", "")
mig_ext_motivo_df_pct_by_plan['motivation'] = mig_ext_motivo_df_pct_by_plan['motivation_choice'].map(choice_labels)
mig_ext_motivo_df_pct_by_plan['percentage'] = mig_ext_motivo_df_pct_by_plan['percentage'] * 100

ynns_labels = {1: "Yes", 0: "No", 99: "Not sure"}

mig_ext_motivo_df_pct_by_plan['mig_ext_plan_12months'] = mig_ext_motivo_df_pct_by_plan['mig_ext_plan_12months'].map(ynns_labels)
mig_ext_motivo_df_pct_by_plan.rename(columns = 
                                     {'mig_ext_plan_12months': "Planning to Move Within 12 Months"}, 
                                     inplace = True)
mig_ext_motivo_df_pct_by_plan.to_csv('./data/mig_ext_motivat_pct_by_plan.csv', index = False)
mig_ext_motivo_df_pct_by_plan.head()

,Planning to Move Within 12 Months,motivation_choice,percentage,motivation
0,No,1,82.726269,Search for a better job
1,Yes,1,83.850932,Search for a better job
2,Not sure,1,84.210526,Search for a better job
3,No,2,29.415011,Unemployment
4,Yes,2,38.819876,Unemployment


# Reasons to migrate internally

In [13]:
cols_to_keep = [f'mig_int_pref_motivo/{n}' for n in range(1, 12)] + ['mig_int_pref_motivo/99']
mig_int_motivo_df = main_df[['mig_int_intention', 'mig_int_plan_12months'] + cols_to_keep]

mig_int_motivo_df_pct = mig_int_motivo_df[mig_int_motivo_df['mig_int_intention'] == 1].mean().to_frame(name = 'percentage')
mig_int_motivo_df_pct.drop(index = ['mig_int_intention', 'mig_int_plan_12months'], inplace = True)
mig_int_motivo_df_pct.reset_index(inplace = True, names = 'motivation_choice')
mig_int_motivo_df_pct['motivation_choice'] = mig_int_motivo_df_pct['motivation_choice'].str.replace("mig_int_pref_motivo/", "")
mig_int_motivo_df_pct['percentage'] = mig_int_motivo_df_pct['percentage'] * 100

choice_labels = {"1": 'Search for a better job',
                 "2": "Family reunification",
                 "3": "Threats/extortion",
                 "4": "Domestic violence",
                 "5": "To find a larger/more comfortable space for family",
                 "6": "For independence or starting own family",
                 "7": "Better academic opportunities",
                 "8": "Deterioration of livelihoods due to natural hazards",
                 "9": "Direct impact of natural hazard",
                 "10": "Due to loss of land (use change)",
                 "11": "Other",
                 "99": "Not sure"
                 }
                 
mig_int_motivo_df_pct['motivation'] = mig_int_motivo_df_pct['motivation_choice'].map(choice_labels)
mig_int_motivo_df_pct.to_csv('./data/mig_int_motivat_pct.csv', index = False)
mig_int_motivo_df_pct


,motivation_choice,percentage,motivation
0,1,83.431953,Search for a better job
1,2,5.071851,Family reunification
2,3,3.719358,Threats/extortion
3,4,0.507185,Domestic violence
4,5,18.681319,To find a larger/more comfortable space for fa...
5,6,6.255283,For independence or starting own family
6,7,8.453085,Better academic opportunities
7,8,4.311074,Deterioration of livelihoods due to natural ha...
8,9,3.296703,Direct impact of natural hazard
9,10,0.507185,Due to loss of land (use change)


In [14]:
mig_int_motivo_df_pct_by_plan = mig_int_motivo_df[mig_int_motivo_df['mig_int_intention'] == 1].groupby('mig_int_plan_12months').mean()

mig_int_motivo_df_pct_by_plan.drop(columns = ['mig_int_intention'], inplace = True)
mig_int_motivo_df_pct_by_plan.reset_index(inplace = True)
mig_int_motivo_df_pct_by_plan = mig_int_motivo_df_pct_by_plan.melt(id_vars = 'mig_int_plan_12months', var_name = 'motivation_choice', value_name = 'percentage')
mig_int_motivo_df_pct_by_plan['motivation_choice'] = mig_int_motivo_df_pct_by_plan['motivation_choice'].str.replace("mig_int_pref_motivo/", "")
mig_int_motivo_df_pct_by_plan['motivation'] = mig_int_motivo_df_pct_by_plan['motivation_choice'].map(choice_labels)
mig_int_motivo_df_pct_by_plan['percentage'] = mig_int_motivo_df_pct_by_plan['percentage'] * 100

mig_int_motivo_df_pct_by_plan['mig_int_plan_12months'] = mig_int_motivo_df_pct_by_plan['mig_int_plan_12months'].map(ynns_labels)
mig_int_motivo_df_pct_by_plan.rename(columns = 
                                     {'mig_int_plan_12months': "Planning to Move Within 12 Months"}, 
                                     inplace = True)
mig_int_motivo_df_pct_by_plan.to_csv('./data/mig_int_motivat_pct_by_plan.csv', index = False)
mig_int_motivo_df_pct_by_plan.head()

,Planning to Move Within 12 Months,motivation_choice,percentage,motivation
0,No,1,84.262948,Search for a better job
1,Yes,1,79.754601,Search for a better job
2,Not sure,1,68.750000,Search for a better job
3,No,2,4.681275,Family reunification
4,Yes,2,6.748466,Family reunification


# Reasons to stay

In [15]:
cols_to_keep = [f'mig_motivo_stay/{n}' for n in range(1, 10)] + ['mig_motivo_stay/99']

mig_stay_motivo_df = main_df[~main_df['mig_motivo_stay'].isna()][cols_to_keep]
mig_stay_motivo_df_pct = mig_stay_motivo_df.mean().to_frame(name = 'percentage')
mig_stay_motivo_df_pct.reset_index(inplace = True, names = 'motivation_choice')
mig_stay_motivo_df_pct['motivation_choice'] = mig_stay_motivo_df_pct['motivation_choice'].str.replace("mig_motivo_stay/", "")
mig_stay_motivo_df_pct['percentage'] = mig_stay_motivo_df_pct['percentage'] * 100

choice_labels = {"1": "Family separation",
                 "2": "Economic stability",
                 "3": "Availability of job opportunities",
                 "4": "The place currently living is safe",
                 "5": "Rootedness/sense of belonging",
                 "6": "Enjoying social status",
                 "7": "Availability of good public services",
                 "8": "Can afford good basic needs",
                 "9": "Other",
                 "99": "Not sure"
                 }
mig_stay_motivo_df_pct['motivation'] = mig_stay_motivo_df_pct['motivation_choice'].map(choice_labels)
mig_stay_motivo_df_pct.to_csv('./data/mig_stay_motivat_pct.csv')
mig_stay_motivo_df_pct


,motivation_choice,percentage,motivation
0,1,65.647627,Family separation
1,2,13.354787,Economic stability
2,3,6.637168,Availability of job opportunities
3,4,40.547064,The place currently living is safe
4,5,22.968624,Rootedness/sense of belonging
5,6,1.488335,Enjoying social status
6,7,4.344328,Availability of good public services
7,8,3.620274,Can afford good basic needs
8,9,12.469831,Other
9,99,3.620274,Not sure


# Convert currency to USD: income, debt, remittance, and expenses

In [16]:
main_df['avg_income_currency'].unique()
# 1 dollars - el salvador 
# 2 lempiras - honduras
# 3 quetzals - guatemala

array([nan,  1.,  3.,  2.])

In [17]:
def fill_na_currency(currency_column):
    print(f"missing (before): {sum(main_df[currency_column].isna())}")
    main_df.loc[(main_df[currency_column].isna()) & (main_df['country'] == 'SLV'),
                    currency_column] = 1
    main_df.loc[(main_df[currency_column].isna()) & (main_df['country'] == 'HND'),
                    currency_column] = 2
    main_df.loc[(main_df[currency_column].isna()) & (main_df['country'] == 'GT'),
                    currency_column] = 3
    print(f"missing (after): {sum(main_df[currency_column].isna())}")
    

def convert_curr_to_usd(val_colname, currency_colname):
    # convert amount to US dollars
    main_df.loc[main_df[currency_colname] == 1, f"{val_colname}_usd"] = main_df[val_colname]
    main_df.loc[main_df[currency_colname] == 2, f"{val_colname}_usd"] = main_df[val_colname] * 0.041
    main_df.loc[main_df[currency_colname] == 3, f"{val_colname}_usd"] = main_df[val_colname] * 0.13

In [18]:
# fill missing currencies based on countries
fill_na_currency('avg_income_currency')
convert_curr_to_usd('avg_income_amount', 'avg_income_currency')
# main_df.loc[(main_df['avg_income_currency'].isna()) & (main_df['country'] == 'SLV'),
#                  'avg_income_currency'] = 1
# main_df.loc[(main_df['avg_income_currency'].isna()) & (main_df['country'] == 'HND'),
#                  'avg_income_currency'] = 2
# main_df.loc[(main_df['avg_income_currency'].isna()) & (main_df['country'] == 'GT'),
#                  'avg_income_currency'] = 3
# print(sum(main_df['avg_income_currency'].isna()))
# convert amount to US dollars
# main_df.loc[main_df['avg_income_currency'] == 1, 'avg_income_amt_dollars'] = main_df['avg_income_amount']
# main_df.loc[main_df['avg_income_currency'] == 2, 'avg_income_amt_dollars'] = main_df['avg_income_amount'] * 0.041
# main_df.loc[main_df['avg_income_currency'] == 3, 'avg_income_amt_dollars'] = main_df['avg_income_amount'] * 0.13

missing (before): 1915
missing (after): 0


In [19]:
fill_na_currency('debt_currency')
convert_curr_to_usd('debt_amount', 'debt_currency')
# print(f"missing (before): {sum(main_df['debt_currency'].isna())}")
# main_df.loc[(main_df['debt_currency'].isna()) & (main_df['country'] == 'SLV'),
#                  'debt_currency'] = 1
# main_df.loc[(main_df['debt_currency'].isna()) & (main_df['country'] == 'HND'),
#                  'debt_currency'] = 2
# main_df.loc[(main_df['debt_currency'].isna()) & (main_df['country'] == 'GT'),
#                  'debt_currency'] = 3
# print(f"missing (after): {sum(main_df['debt_currency'].isna())}")

# main_df.loc[main_df['debt_currency'] == 1, 'debt_amount_dollars'] = main_df['debt_amount']
# main_df.loc[main_df['debt_currency'] == 2, 'debt_amount_dollars'] = main_df['debt_amount'] * 0.041
# main_df.loc[main_df['debt_currency'] == 3, 'debt_amount_dollars'] = main_df['debt_amount'] * 0.13

missing (before): 3924
missing (after): 0


In [20]:
print(main_df['remesa_currency'].unique())
main_df['remesa_currency'].value_counts()

[nan  1.  2.  3.]


1.0    607
3.0    248
2.0    172
Name: remesa_currency, dtype: int64

In [21]:
fill_na_currency('remesa_currency')
convert_curr_to_usd('remesa_amount', 'remesa_currency')

missing (before): 3971
missing (after): 0


In [22]:
print(main_df['exp_currency'].unique())
main_df['exp_currency'].value_counts()

[1 2 3]


3    1726
1    1705
2    1567
Name: exp_currency, dtype: int64

In [23]:
print(main_df['exp_6months_currency'].unique())
main_df['exp_6months_currency'].value_counts()

[1 2 3]


3    1732
1    1690
2    1576
Name: exp_6months_currency, dtype: int64

In [24]:
exp_col_list = ['exp_monthly_food', 'exp_monthly_drinking_water', 'exp_monthly_water',
                'exp_monthly_rent', 'exp_monthly_electric_power', 'exp_monthly_hygiene',
                'exp_monthly_transport', 'exp_monthly_communication', 'exp_monthly_fuel',
                'exp_monthly_other']
# fill_na_currency('remesa_currency')
for feature in exp_col_list:
    convert_curr_to_usd(feature, 'exp_currency')
    
exp6m_col_list = ['exp_6months_health', 'exp_6months_clothing', 'exp_6months_education',
                  'exp_6months_debt', 'exp_6months_social', 'exp_6months_inputs',
                  'exp_6months_savings', 'exp_6months_repairhouse']
for feature in exp6m_col_list:
    convert_curr_to_usd(feature, 'exp_6months_currency')


# Roster data: summarize and merge with main_df

In [25]:
roster_df = pd.read_csv(root + "dataset-1_central-american-survey/hh_roster.csv", index_col = 0)
roster_df.head(3)

,hh_sex,hh_age,cond_migracion,escolaridad,work_30days,mig_ocupacion,orig_ocupacion,student_status,modalidad_educacion,no_escuela_why,...,_submission__id,_submission__uuid,_submission__submission_time,_submission__validation_status,_submission__notes,_submission__status,_submission__submitted_by,_submission__tags,country,migrated_yn
0,2,22,1,4.0,0.0,5.0,NaN,88.0,NaN,NaN,...,174925765,ccfb10fe-7afa-48d8-bea2-08c40c24d7b6,2021-04-30T00:34:53,NaN,[],submitted_via_web,NaN,[],GT,0
1,1,29,1,5.0,0.0,5.0,NaN,88.0,NaN,NaN,...,174925765,ccfb10fe-7afa-48d8-bea2-08c40c24d7b6,2021-04-30T00:34:53,NaN,[],submitted_via_web,NaN,[],GT,0
2,2,59,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,...,174925765,ccfb10fe-7afa-48d8-bea2-08c40c24d7b6,2021-04-30T00:34:53,NaN,[],submitted_via_web,NaN,[],GT,0


### breastfeeding and prenancies

In [26]:
print(roster_df['breastfeeding_yn'].value_counts())
print(roster_df['pregnant_yn'].value_counts())

0.0     6397
1.0      544
99.0       9
Name: breastfeeding_yn, dtype: int64
0.0     6702
1.0      232
99.0      16
Name: pregnant_yn, dtype: int64


In [27]:
# change 99 (not sure) to 0 (no)
roster_df.loc[roster_df['breastfeeding_yn'] == 99, 'breastfeeding_yn'] = 0
roster_df.loc[roster_df['pregnant_yn'] == 99, 'pregnant_yn'] = 0

roster_df.loc[roster_df['escolaridad'] == 99, 'escolaridad'] = 0 # highest level of education

roster_df['is_child'] = roster_df['hh_age'] < 15 # child is defined as age under 15
roster_df['is_elder'] = roster_df['hh_age'] >= 65

roster_df['vulnerability_disability'] = roster_df['vulnerability/1'] == 1
roster_df['vulnerability_chronic'] = roster_df['vulnerability/2'] == 1

In [28]:
hh_composition = roster_df.groupby('_submission__uuid').agg({'escolaridad' : 'max',
                                            'hh_age': 'mean',
                                            'pregnant_yn': 'max',
                                            'breastfeeding_yn': 'max',
                                            'is_child': 'sum',
                                            'is_elder': 'sum',
                                            'vulnerability_disability': 'sum',
                                            'vulnerability_chronic': 'sum'})
hh_composition.reset_index(inplace = True)
hh_composition.rename(columns = {'escolaridad': 'highest_educ',
                                 'hh_age': 'avg_age',
                                 'pregnant_yn': 'has_pregnant',
                                 'breastfeeding_yn': 'has_breastfeeding',
                                 'is_child': 'num_child_15',
                                 'is_elder': 'num_elderly_65',
                                 'vulnerability_disability': 'num_disability',
                                 'vulnerability_chronic': 'num_chronic'
                                 }, inplace = True)
hh_composition.head()

,_submission__uuid,highest_educ,avg_age,has_pregnant,has_breastfeeding,num_child_15,num_elderly_65,num_disability,num_chronic
0,001041a4-4559-4f59-843d-ffb0a9e75e22,3.0,31.000000,0.0,0.0,1,0,0,0
1,001e7f5c-2926-4612-93dc-9adbfb63917e,8.0,45.200000,0.0,0.0,0,1,0,0
2,0026f523-33ef-44a1-a528-fa73b18f3502,3.0,61.500000,NaN,NaN,0,0,0,1
3,00302df2-b730-4ba8-ba6a-8ef2699c7c16,6.0,72.500000,NaN,NaN,0,2,0,1
4,003f2628-766c-4585-8024-1df0a38116b5,7.0,31.833333,0.0,0.0,1,2,0,0


In [29]:
main_df = main_df.merge(hh_composition, how = 'left',
                                  left_on = '_uuid', right_on = '_submission__uuid')
main_df

,start,end,note_confidenciality,organizacion,survey_pid,country,departamento,municipio,neighborhood,rural_urban,...,exp_6months_repairhouse_usd,_submission__uuid,highest_educ,avg_age,has_pregnant,has_breastfeeding,num_child_15,num_elderly_65,num_disability,num_chronic
0,2021-05-13T09:29:28.324-06:00,2021-05-13T09:59:31.341-06:00,NaN,1,34,SLV,1193,15748,Condominios delgado,2,...,60.0,3204a982-fb39-426b-9ea2-c957d8b5e222,8.0,31.333333,0.0,0.0,1.0,0.0,0.0,2.0
1,2021-05-13T08:44:14.642-06:00,2021-05-13T09:05:10.406-06:00,NaN,1,34,SLV,1193,15748,Condominios santa maria,2,...,0.0,6f714d43-c5d2-4ad6-900d-06004e2fb241,8.0,44.666667,NaN,NaN,0.0,0.0,0.0,1.0
2,2021-05-13T08:10:59.086-06:00,2021-05-13T08:43:16.923-06:00,NaN,1,34,SLV,1193,15748,Condominios santa maria,2,...,0.0,da9eaf26-c33c-4b32-8a95-d0bdd2a2f4ef,6.0,55.250000,0.0,0.0,0.0,1.0,0.0,2.0
3,2021-05-13T09:53:37.283-06:00,2021-05-13T10:22:11.248-06:00,NaN,1,35,SLV,1193,15748,Condominio delgado,2,...,0.0,5de19d96-c5a8-44ca-81be-619abb979b9e,5.0,22.600000,0.0,0.0,2.0,0.0,0.0,0.0
4,2021-05-13T09:31:16.086-06:00,2021-05-13T09:53:06.037-06:00,NaN,1,35,SLV,1193,15748,Condominio delgado,2,...,0.0,858dbb65-055d-448e-ac10-5be3d4a8f866,5.0,16.666667,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,2021-04-29T09:57:28.139-06:00,2021-04-29T10:31:21.208-06:00,NaN,5,30,GT,GT16,GT1601,Barrio canton zona 8,2,...,0.0,08aa6f28-d9d5-41f3-8e71-910954f2fdc0,7.0,35.200000,0.0,0.0,1.0,1.0,0.0,0.0
4994,2021-04-29T10:44:36.260-06:00,2021-04-29T11:07:19.484-06:00,NaN,5,30,GT,GT16,GT1601,Barrio canton zona 8,2,...,0.0,1a80066b-f54e-45b3-b740-00df77b72d14,4.0,47.666667,0.0,0.0,0.0,1.0,0.0,0.0
4995,2021-04-29T14:14:14.013-06:00,2021-04-29T14:39:45.356-06:00,NaN,5,36,GT,GT20,GT2003,San juan ermita,2,...,0.0,b5826f04-558c-4bd2-adaf-7bf45c8fe168,5.0,31.500000,0.0,0.0,0.0,0.0,1.0,0.0
4996,2021-04-29T14:40:13.473-06:00,2021-04-29T15:04:26.498-06:00,NaN,5,36,GT,GT20,GT2003,Sanjuan ermita,2,...,0.0,3342c82a-f4fd-4753-8503-2179bc6712ed,1.0,55.000000,NaN,NaN,0.0,1.0,2.0,0.0


In [31]:
educ_labels = {1: 'No education',
               2: 'Preschool',
               3: "Primary",
               4: "High school",
               5: "General bac.",
               6: "Technical bac.",
               7: "Uni educ technician",
               8: "Bachelor's",
               9: "Postgraduate"}
            #    0: "Not sure"}
main_df['highest_educ'] = main_df['highest_educ'].map(educ_labels)

# Cost of migration

In [32]:
mig_ext_df = pd.read_csv(root + "dataset-1_central-american-survey/mig_ext_roster.csv", index_col = 0)
print(f"Number of records: {mig_ext_df.shape[0]}")
mig_ext_df

Number of records: 1624


,mig_ext_sex,mig_ext_age,mig_ext_motivo,mig_ext_motivo/1,mig_ext_motivo/2,mig_ext_motivo/3,mig_ext_motivo/4,mig_ext_motivo/5,mig_ext_motivo/6,mig_ext_motivo/7,...,_submission__tags,country,mig_ext_finance_group/1,mig_ext_finance_group/2,mig_ext_finance_group/3,mig_ext_medio_group/1,mig_ext_medio_group/2,mig_ext_medio_group/3,cost_per_attempt,migrated_yn
0,1,20,1 2 6,1,1,0,0,0,1,0,...,[],GT,0,1,0,0,1,0,NaN,1
1,2,35,1 8,1,0,0,0,0,0,0,...,[],HND,0,1,0,0,0,1,800.000000,1
2,1,44,1 2 6,1,1,0,0,0,1,0,...,[],HND,0,0,1,0,0,1,NaN,1
3,2,29,1 8,1,0,0,0,0,0,0,...,[],GT,1,0,0,0,1,0,3466.666667,1
4,2,35,1,1,0,0,0,0,0,0,...,[],SLV,0,1,0,0,1,0,5500.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,1,42,1 2 8,1,1,0,0,0,0,0,...,[],SLV,0,1,0,0,1,0,NaN,1
1620,2,30,1 6 8,1,0,0,0,0,1,0,...,[],HND,0,1,0,0,0,1,NaN,1
1621,2,30,1 6,1,0,0,0,0,1,0,...,[],HND,0,0,0,0,0,1,NaN,1
1622,1,26,1,1,0,0,0,0,0,0,...,[],HND,0,1,0,1,0,0,NaN,1


In [33]:
print(f"Number of N/A currency: {sum(mig_ext_df['mig_ext_cost_currency'].isna())}")
print(f"Number of N/A cost_total: {sum(mig_ext_df['mig_ext_cost_total'].isna())}")
mig_ext_df['mig_ext_cost_currency'].value_counts()

Number of N/A currency: 807
Number of N/A cost_total: 809


1.0    369
3.0    342
2.0    106
Name: mig_ext_cost_currency, dtype: int64

In [34]:
print(sum(mig_ext_df[~mig_ext_df['mig_ext_cost_total'].isna()]['mig_ext_cost_currency'].isna()))

mig_ext_df.loc[(mig_ext_df['mig_ext_cost_currency'].isna()) & (~mig_ext_df['mig_ext_cost_total'].isna()) \
                & (mig_ext_df['country'] == 'SLV'),
                 'mig_ext_cost_currency'] = 1
mig_ext_df.loc[(mig_ext_df['mig_ext_cost_currency'].isna()) & (~mig_ext_df['mig_ext_cost_total'].isna()) & \
                (mig_ext_df['country'] == 'HND'),
                 'mig_ext_cost_currency'] = 2
mig_ext_df.loc[(mig_ext_df['mig_ext_cost_currency'].isna()) & (~mig_ext_df['mig_ext_cost_total'].isna()) & \
                (mig_ext_df['country'] == 'GT'),
                 'mig_ext_cost_currency'] = 3
print(sum(mig_ext_df[~mig_ext_df['mig_ext_cost_total'].isna()]['mig_ext_cost_currency'].isna()))


3
0


In [35]:
mig_ext_df.loc[mig_ext_df['mig_ext_cost_currency'] == 1, 'mig_ext_cost_total_usd'] = mig_ext_df['mig_ext_cost_total']
mig_ext_df.loc[mig_ext_df['mig_ext_cost_currency'] == 2, 'mig_ext_cost_total_usd'] = mig_ext_df['mig_ext_cost_total'] * 0.041
mig_ext_df.loc[mig_ext_df['mig_ext_cost_currency'] == 3, 'mig_ext_cost_total_usd'] = mig_ext_df['mig_ext_cost_total'] * 0.13

In [81]:
mig_ext_df.to_csv('./data/mig_ext_cleaned.csv', index = False)

In [83]:
main_df.to_csv('./data/main_cleaned.csv', index = False)

# Expenditure categories

In [37]:
main_df.head()

,start,end,note_confidenciality,organizacion,survey_pid,country,departamento,municipio,neighborhood,rural_urban,...,exp_6months_repairhouse_usd,_submission__uuid,highest_educ,avg_age,has_pregnant,has_breastfeeding,num_child_15,num_elderly_65,num_disability,num_chronic
0,2021-05-13T09:29:28.324-06:00,2021-05-13T09:59:31.341-06:00,NaN,1,34,SLV,1193,15748,Condominios delgado,2,...,60.0,3204a982-fb39-426b-9ea2-c957d8b5e222,Bachelor's,31.333333,0.0,0.0,1.0,0.0,0.0,2.0
1,2021-05-13T08:44:14.642-06:00,2021-05-13T09:05:10.406-06:00,NaN,1,34,SLV,1193,15748,Condominios santa maria,2,...,0.0,6f714d43-c5d2-4ad6-900d-06004e2fb241,Bachelor's,44.666667,NaN,NaN,0.0,0.0,0.0,1.0
2,2021-05-13T08:10:59.086-06:00,2021-05-13T08:43:16.923-06:00,NaN,1,34,SLV,1193,15748,Condominios santa maria,2,...,0.0,da9eaf26-c33c-4b32-8a95-d0bdd2a2f4ef,Technical bac.,55.250000,0.0,0.0,0.0,1.0,0.0,2.0
3,2021-05-13T09:53:37.283-06:00,2021-05-13T10:22:11.248-06:00,NaN,1,35,SLV,1193,15748,Condominio delgado,2,...,0.0,5de19d96-c5a8-44ca-81be-619abb979b9e,General bac.,22.600000,0.0,0.0,2.0,0.0,0.0,0.0
4,2021-05-13T09:31:16.086-06:00,2021-05-13T09:53:06.037-06:00,NaN,1,35,SLV,1193,15748,Condominio delgado,2,...,0.0,858dbb65-055d-448e-ac10-5be3d4a8f866,General bac.,16.666667,0.0,0.0,2.0,0.0,0.0,0.0


In [40]:
main_df['exp_6months_currency'].unique()

array([1, 2, 3])

In [41]:
for col in ['exp_monthly_food', 'exp_monthly_drinking_water', 'exp_monthly_water',
            'exp_monthly_rent', 'exp_monthly_electric_power', 'exp_monthly_hygiene',
            'exp_monthly_transport', 'exp_monthly_communication', 'exp_monthly_fuel',
            'exp_monthly_other']:
    convert_curr_to_usd(col, "exp_currency")
    
for col in ['exp_6months_health', 'exp_6months_clothing', 'exp_6months_education',
            'exp_6months_debt', 'exp_6months_social', 'exp_6months_inputs', 
            'exp_6months_savings', 'exp_6months_repairhouse']:
    convert_curr_to_usd(col, "exp_6months_currency")

In [44]:
exp_6m = ['health', 'clothing', 'education',
            'debt', 'social', 'inputs', 
            'savings', 'repairhouse']
# convert to monthly
for col in exp_6m:
    main_df[f'exp_monthly_{col}_usd'] = main_df[f"exp_6months_{col}_usd"] / 6


/var/folders/kl/zzq83xsx65s2p501hz1mwvlw0000gn/T/ipykernel_12364/3009202470.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  main_df[f'exp_monthly_{col}_usd'] = main_df[f"exp_6months_{col}_usd"] / 6
/var/folders/kl/zzq83xsx65s2p501hz1mwvlw0000gn/T/ipykernel_12364/3009202470.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  main_df[f'exp_monthly_{col}_usd'] = main_df[f"exp_6months_{col}_usd"] / 6
/var/folders/kl/zzq83xsx65s2p501hz1mwvlw0000gn/T/ipykernel_12364/3009202470.py:6: PerformanceWarning: DataFrame is highly fragment

In [49]:
main_df['remittances_yn'].value_counts()

0     3532
1     1447
99      19
Name: remittances_yn, dtype: int64

In [ ]:
exp_cols = ['exp_monthly_food', 'exp_monthly_drinking_water', 'exp_monthly_water',
            'exp_monthly_rent', 'exp_monthly_electric_power', 'exp_monthly_hygiene',
            'exp_monthly_transport', 'exp_monthly_communication', 'exp_monthly_fuel',
            'exp_monthly_other']
exp_cols = [n + "_usd" for n in exp_cols]
exp_cols = exp_cols + [f"exp_monthly_{x}_usd" for x in exp_6m]

main_df.groupby(['remittances_yn'])[exp_cols]]